# 透過機器學習預測「電信業」顧客流失率-03模型訓練與參數調整

# 作者：林蔚恩(臺灣行銷研究特邀作者)、徐佳靖(臺灣行銷研究特邀作者)、張家穎(臺灣行銷研究特邀作者)

# 原始資料請見[本連結](https://www.kaggle.com/blastchar/telco-customer-churn)，下載下來後將pd.read_csv()內的路徑改為資料集的路徑即可

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # For creating plots
import matplotlib.ticker as mtick # For specifying the axes tick format 
import matplotlib.pyplot as plt
import sklearn
import scipy
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
telecom_cust = pd.read_csv('Telco-Customer-Churn.csv')

In [3]:
telecom_cust.TotalCharges = pd.to_numeric(telecom_cust.TotalCharges, errors='coerce')
telecom_cust.isnull().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [4]:
#將遺漏值移除
telecom_cust.dropna(inplace = True)

In [5]:
#建立標準化函式
def standardization(data):
    mu = np.mean(data, axis=0)
    sigma = np.std(data, axis=0)
    return (data - mu) / sigma

In [6]:
#將總費用進行標準化處理
telecom_cust['TotalCharges']= standardization(telecom_cust['TotalCharges'])

In [7]:
df2 = telecom_cust.iloc[:,1:]

In [8]:
df2['Churn'].replace(to_replace='Yes', value=1, inplace=True)
df2['Churn'].replace(to_replace='No',  value=0, inplace=True)

In [9]:
#將類別變數數轉換為虛擬變數
df_dummies = pd.get_dummies(df2)

In [10]:
#將流失率設為應變數
x= df_dummies.drop(columns=['Churn'])
y= df_dummies ['Churn']

In [11]:
#切分訓練資料與測試資料
x_train, x_test, y_train, y_test = train_test_split(x,y,stratify=y, test_size = 0.2, random_state = 0)

# 以下為決策樹（Decision Tree, DT）模型訓練

In [12]:
classifier_dt = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier_dt.fit(x_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [13]:
y_pred_dt = classifier_dt.predict(x_test)

# 以下為支持向量機（Support Vector Mechine, SVM）模型訓練

In [14]:
classifier_svm = SVC(kernel = 'linear', random_state = 0)
classifier_svm.fit(x_train, y_train)

SVC(kernel='linear', random_state=0)

In [15]:
y_pred_svm = classifier_svm.predict(x_test)

# 以下為羅吉斯迴歸（Logistic Regression）模型訓練與調參

In [16]:
model_reg = LogisticRegression(random_state = 0)
model_reg.fit(x_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=0)

In [17]:
y_pred_reg = model_reg.predict(x_test)

In [19]:
grid = {
    'C': np.linspace(0.0,0.5,100)
     , 'solver': ['lbfgs','liblinear','sag']
}
model=LogisticRegression(penalty='l2',random_state=100, max_iter=10000, tol=10,class_weight='balanced')
gs = GridSearchCV(model, grid, scoring='accuracy', cv=3,verbose=10)
gs.fit(x_train,y_train)

print ('gs.best_score_:', gs.best_score_)
print("tuned hpyerparameters :(best parameters) ",gs.best_params_)

Fitting 3 folds for each of 300 candidates, totalling 900 fits
[CV 1/3; 1/300] START C=0.0, solver=lbfgs.......................................
[CV 1/3; 1/300] END ..........C=0.0, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/3; 1/300] START C=0.0, solver=lbfgs.......................................
[CV 2/3; 1/300] END ..........C=0.0, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/3; 1/300] START C=0.0, solver=lbfgs.......................................
[CV 3/3; 1/300] END ..........C=0.0, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/3; 2/300] START C=0.0, solver=liblinear...................................
[CV 1/3; 2/300] END ......C=0.0, solver=liblinear;, score=nan total time=   0.0s
[CV 2/3; 2/300] START C=0.0, solver=liblinear...................................
[CV 2/3; 2/300] END ......C=0.0, solver=liblinear;, score=nan total time=   0.0s
[CV 3/3; 2/300] START C=0.0, solver=liblinear...................................
[CV 3/3; 2/300] END ......C=0.0, solver=liblin

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:811: RuntimeWarning: divide by zero encountered in double_scalars
  args=(X, target, 1.0 / C, sample_weight),
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:122: RuntimeWarning: invalid value encountered in double_scalars
  out = -np.sum(sample_weight * log_logistic(yz)) + 0.5 * alpha * np.dot(w, w)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:127: RuntimeWarning: invalid value encountered in multiply
  grad[:n_features] = safe_sparse_dot(X.T, z0) + alpha * w
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:811: RuntimeWarning: divide by zero encountered in double_scalars
  args=(X, target, 1.0 / C, sample_weight),
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-p

[CV 3/3; 4/300] END C=0.005050505050505051, solver=lbfgs;, score=0.741 total time=   0.0s
[CV 1/3; 5/300] START C=0.005050505050505051, solver=liblinear..................
[CV 1/3; 5/300] END C=0.005050505050505051, solver=liblinear;, score=0.734 total time=   0.0s
[CV 2/3; 5/300] START C=0.005050505050505051, solver=liblinear..................
[CV 2/3; 5/300] END C=0.005050505050505051, solver=liblinear;, score=0.734 total time=   0.0s
[CV 3/3; 5/300] START C=0.005050505050505051, solver=liblinear..................
[CV 3/3; 5/300] END C=0.005050505050505051, solver=liblinear;, score=0.734 total time=   0.0s
[CV 1/3; 6/300] START C=0.005050505050505051, solver=sag........................
[CV 1/3; 6/300] END C=0.005050505050505051, solver=sag;, score=0.414 total time=   0.0s
[CV 2/3; 6/300] START C=0.005050505050505051, solver=sag........................
[CV 2/3; 6/300] END C=0.005050505050505051, solver=sag;, score=0.497 total time=   0.0s
[CV 3/3; 6/300] START C=0.005050505050505051, s

[CV 1/3; 22/300] END C=0.03535353535353536, solver=lbfgs;, score=0.769 total time=   0.1s
[CV 2/3; 22/300] START C=0.03535353535353536, solver=lbfgs......................
[CV 2/3; 22/300] END C=0.03535353535353536, solver=lbfgs;, score=0.740 total time=   0.1s
[CV 3/3; 22/300] START C=0.03535353535353536, solver=lbfgs......................
[CV 3/3; 22/300] END C=0.03535353535353536, solver=lbfgs;, score=0.746 total time=   0.0s
[CV 1/3; 23/300] START C=0.03535353535353536, solver=liblinear..................
[CV 1/3; 23/300] END C=0.03535353535353536, solver=liblinear;, score=0.734 total time=   0.0s
[CV 2/3; 23/300] START C=0.03535353535353536, solver=liblinear..................
[CV 2/3; 23/300] END C=0.03535353535353536, solver=liblinear;, score=0.734 total time=   0.0s
[CV 3/3; 23/300] START C=0.03535353535353536, solver=liblinear..................
[CV 3/3; 23/300] END C=0.03535353535353536, solver=liblinear;, score=0.734 total time=   0.0s
[CV 1/3; 24/300] START C=0.0353535353535353

[CV 3/3; 40/300] END C=0.06565656565656566, solver=lbfgs;, score=0.742 total time=   0.1s
[CV 1/3; 41/300] START C=0.06565656565656566, solver=liblinear..................
[CV 1/3; 41/300] END C=0.06565656565656566, solver=liblinear;, score=0.734 total time=   0.0s
[CV 2/3; 41/300] START C=0.06565656565656566, solver=liblinear..................
[CV 2/3; 41/300] END C=0.06565656565656566, solver=liblinear;, score=0.734 total time=   0.0s
[CV 3/3; 41/300] START C=0.06565656565656566, solver=liblinear..................
[CV 3/3; 41/300] END C=0.06565656565656566, solver=liblinear;, score=0.734 total time=   0.0s
[CV 1/3; 42/300] START C=0.06565656565656566, solver=sag........................
[CV 1/3; 42/300] END C=0.06565656565656566, solver=sag;, score=0.437 total time=   0.0s
[CV 2/3; 42/300] START C=0.06565656565656566, solver=sag........................
[CV 2/3; 42/300] END C=0.06565656565656566, solver=sag;, score=0.476 total time=   0.0s
[CV 3/3; 42/300] START C=0.06565656565656566, s

...

[CV 2/3; 234/300] END C=0.3888888888888889, solver=sag;, score=0.474 total time=   0.0s
[CV 3/3; 234/300] START C=0.3888888888888889, solver=sag........................
[CV 3/3; 234/300] END C=0.3888888888888889, solver=sag;, score=0.759 total time=   0.0s
[CV 1/3; 235/300] START C=0.393939393939394, solver=lbfgs.......................
[CV 1/3; 235/300] END C=0.393939393939394, solver=lbfgs;, score=0.765 total time=   0.1s
[CV 2/3; 235/300] START C=0.393939393939394, solver=lbfgs.......................
[CV 2/3; 235/300] END C=0.393939393939394, solver=lbfgs;, score=0.733 total time=   0.1s
[CV 3/3; 235/300] START C=0.393939393939394, solver=lbfgs.......................
[CV 3/3; 235/300] END C=0.393939393939394, solver=lbfgs;, score=0.742 total time=   0.1s
[CV 1/3; 236/300] START C=0.393939393939394, solver=liblinear...................
[CV 1/3; 236/300] END C=0.393939393939394, solver=liblinear;, score=0.734 total time=   0.0s
[CV 2/3; 236/300] START C=0.393939393939394, solver=libline

[CV 2/3; 250/300] END C=0.4191919191919192, solver=lbfgs;, score=0.729 total time=   0.1s
[CV 3/3; 250/300] START C=0.4191919191919192, solver=lbfgs......................
[CV 3/3; 250/300] END C=0.4191919191919192, solver=lbfgs;, score=0.742 total time=   0.1s
[CV 1/3; 251/300] START C=0.4191919191919192, solver=liblinear..................
[CV 1/3; 251/300] END C=0.4191919191919192, solver=liblinear;, score=0.734 total time=   0.0s
[CV 2/3; 251/300] START C=0.4191919191919192, solver=liblinear..................
[CV 2/3; 251/300] END C=0.4191919191919192, solver=liblinear;, score=0.734 total time=   0.0s
[CV 3/3; 251/300] START C=0.4191919191919192, solver=liblinear..................
[CV 3/3; 251/300] END C=0.4191919191919192, solver=liblinear;, score=0.734 total time=   0.0s
[CV 1/3; 252/300] START C=0.4191919191919192, solver=sag........................
[CV 1/3; 252/300] END C=0.4191919191919192, solver=sag;, score=0.437 total time=   0.0s
[CV 2/3; 252/300] START C=0.4191919191919192,

[CV 2/3; 268/300] END C=0.44949494949494956, solver=lbfgs;, score=0.733 total time=   0.1s
[CV 3/3; 268/300] START C=0.44949494949494956, solver=lbfgs.....................
[CV 3/3; 268/300] END C=0.44949494949494956, solver=lbfgs;, score=0.743 total time=   0.1s
[CV 1/3; 269/300] START C=0.44949494949494956, solver=liblinear.................
[CV 1/3; 269/300] END C=0.44949494949494956, solver=liblinear;, score=0.734 total time=   0.0s
[CV 2/3; 269/300] START C=0.44949494949494956, solver=liblinear.................
[CV 2/3; 269/300] END C=0.44949494949494956, solver=liblinear;, score=0.734 total time=   0.0s
[CV 3/3; 269/300] START C=0.44949494949494956, solver=liblinear.................
[CV 3/3; 269/300] END C=0.44949494949494956, solver=liblinear;, score=0.734 total time=   0.0s
[CV 1/3; 270/300] START C=0.44949494949494956, solver=sag.......................
[CV 1/3; 270/300] END C=0.44949494949494956, solver=sag;, score=0.437 total time=   0.0s
[CV 2/3; 270/300] START C=0.44949494949

[CV 1/3; 286/300] END C=0.47979797979797983, solver=lbfgs;, score=0.769 total time=   0.1s
[CV 2/3; 286/300] START C=0.47979797979797983, solver=lbfgs.....................
[CV 2/3; 286/300] END C=0.47979797979797983, solver=lbfgs;, score=0.733 total time=   0.1s
[CV 3/3; 286/300] START C=0.47979797979797983, solver=lbfgs.....................
[CV 3/3; 286/300] END C=0.47979797979797983, solver=lbfgs;, score=0.741 total time=   0.1s
[CV 1/3; 287/300] START C=0.47979797979797983, solver=liblinear.................
[CV 1/3; 287/300] END C=0.47979797979797983, solver=liblinear;, score=0.734 total time=   0.0s
[CV 2/3; 287/300] START C=0.47979797979797983, solver=liblinear.................
[CV 2/3; 287/300] END C=0.47979797979797983, solver=liblinear;, score=0.734 total time=   0.0s
[CV 3/3; 287/300] START C=0.47979797979797983, solver=liblinear.................
[CV 3/3; 287/300] END C=0.47979797979797983, solver=liblinear;, score=0.734 total time=   0.0s
[CV 1/3; 288/300] START C=0.479797979

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
9 fits failed out of a total of 900.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1614, in fit
    for class_, warm_start_coef_ in zip(classes_, warm_start_coef)
  File "/Library/Frameworks/Python

# 以下為 隨機森林（Random Forest） 模型訓練與調參

In [20]:
classifier = RandomForestClassifier()
classifier.fit(x_train, y_train)

RandomForestClassifier()

In [21]:
y_pred = classifier.predict(x_test)

In [22]:
grid = {
    'n_estimators': np.arange(1,50,1)
     ,'criterion': ['gini','entropy']
     ,'max_depth': np.linspace(1,11,11)
     ,'max_features': ['auto', 'log2']
}
model=RandomForestClassifier(random_state=10)
gs = GridSearchCV(model, grid, scoring='accuracy', cv=3,verbose=10)
gs.fit(x_train,y_train)

print ('gs.best_score_:', gs.best_score_)
print("tuned hpyerparameters :(best parameters) ",gs.best_params_)

Fitting 3 folds for each of 2156 candidates, totalling 6468 fits
[CV 1/3; 1/2156] START criterion=gini, max_depth=1.0, max_features=auto, n_estimators=1
[CV 1/3; 1/2156] END criterion=gini, max_depth=1.0, max_features=auto, n_estimators=1;, score=0.734 total time=   0.0s
[CV 2/3; 1/2156] START criterion=gini, max_depth=1.0, max_features=auto, n_estimators=1
[CV 2/3; 1/2156] END criterion=gini, max_depth=1.0, max_features=auto, n_estimators=1;, score=0.734 total time=   0.0s
[CV 3/3; 1/2156] START criterion=gini, max_depth=1.0, max_features=auto, n_estimators=1
[CV 3/3; 1/2156] END criterion=gini, max_depth=1.0, max_features=auto, n_estimators=1;, score=0.725 total time=   0.0s
[CV 1/3; 2/2156] START criterion=gini, max_depth=1.0, max_features=auto, n_estimators=2
[CV 1/3; 2/2156] END criterion=gini, max_depth=1.0, max_features=auto, n_estimators=2;, score=0.734 total time=   0.0s
[CV 2/3; 2/2156] START criterion=gini, max_depth=1.0, max_features=auto, n_estimators=2
[CV 2/3; 2/2156] EN

[CV 3/3; 14/2156] END criterion=gini, max_depth=1.0, max_features=auto, n_estimators=14;, score=0.734 total time=   0.1s
[CV 1/3; 15/2156] START criterion=gini, max_depth=1.0, max_features=auto, n_estimators=15
[CV 1/3; 15/2156] END criterion=gini, max_depth=1.0, max_features=auto, n_estimators=15;, score=0.734 total time=   0.1s
[CV 2/3; 15/2156] START criterion=gini, max_depth=1.0, max_features=auto, n_estimators=15
[CV 2/3; 15/2156] END criterion=gini, max_depth=1.0, max_features=auto, n_estimators=15;, score=0.734 total time=   0.1s
[CV 3/3; 15/2156] START criterion=gini, max_depth=1.0, max_features=auto, n_estimators=15
[CV 3/3; 15/2156] END criterion=gini, max_depth=1.0, max_features=auto, n_estimators=15;, score=0.734 total time=   0.0s
[CV 1/3; 16/2156] START criterion=gini, max_depth=1.0, max_features=auto, n_estimators=16
[CV 1/3; 16/2156] END criterion=gini, max_depth=1.0, max_features=auto, n_estimators=16;, score=0.734 total time=   0.1s
[CV 2/3; 16/2156] START criterion=g

[CV 2/3; 28/2156] END criterion=gini, max_depth=1.0, max_features=auto, n_estimators=28;, score=0.734 total time=   0.1s
[CV 3/3; 28/2156] START criterion=gini, max_depth=1.0, max_features=auto, n_estimators=28
[CV 3/3; 28/2156] END criterion=gini, max_depth=1.0, max_features=auto, n_estimators=28;, score=0.734 total time=   0.1s
[CV 1/3; 29/2156] START criterion=gini, max_depth=1.0, max_features=auto, n_estimators=29
[CV 1/3; 29/2156] END criterion=gini, max_depth=1.0, max_features=auto, n_estimators=29;, score=0.734 total time=   0.1s
[CV 2/3; 29/2156] START criterion=gini, max_depth=1.0, max_features=auto, n_estimators=29
[CV 2/3; 29/2156] END criterion=gini, max_depth=1.0, max_features=auto, n_estimators=29;, score=0.734 total time=   0.1s
[CV 3/3; 29/2156] START criterion=gini, max_depth=1.0, max_features=auto, n_estimators=29
[CV 3/3; 29/2156] END criterion=gini, max_depth=1.0, max_features=auto, n_estimators=29;, score=0.734 total time=   0.1s
[CV 1/3; 30/2156] START criterion=g

[CV 3/3; 41/2156] END criterion=gini, max_depth=1.0, max_features=auto, n_estimators=41;, score=0.734 total time=   0.1s
[CV 1/3; 42/2156] START criterion=gini, max_depth=1.0, max_features=auto, n_estimators=42
[CV 1/3; 42/2156] END criterion=gini, max_depth=1.0, max_features=auto, n_estimators=42;, score=0.734 total time=   0.1s
[CV 2/3; 42/2156] START criterion=gini, max_depth=1.0, max_features=auto, n_estimators=42
[CV 2/3; 42/2156] END criterion=gini, max_depth=1.0, max_features=auto, n_estimators=42;, score=0.734 total time=   0.1s
[CV 3/3; 42/2156] START criterion=gini, max_depth=1.0, max_features=auto, n_estimators=42
[CV 3/3; 42/2156] END criterion=gini, max_depth=1.0, max_features=auto, n_estimators=42;, score=0.734 total time=   0.2s
[CV 1/3; 43/2156] START criterion=gini, max_depth=1.0, max_features=auto, n_estimators=43
[CV 1/3; 43/2156] END criterion=gini, max_depth=1.0, max_features=auto, n_estimators=43;, score=0.734 total time=   0.2s
[CV 2/3; 43/2156] START criterion=g

[CV 1/3; 57/2156] END criterion=gini, max_depth=1.0, max_features=log2, n_estimators=8;, score=0.734 total time=   0.0s
[CV 2/3; 57/2156] START criterion=gini, max_depth=1.0, max_features=log2, n_estimators=8
[CV 2/3; 57/2156] END criterion=gini, max_depth=1.0, max_features=log2, n_estimators=8;, score=0.734 total time=   0.0s
[CV 3/3; 57/2156] START criterion=gini, max_depth=1.0, max_features=log2, n_estimators=8
[CV 3/3; 57/2156] END criterion=gini, max_depth=1.0, max_features=log2, n_estimators=8;, score=0.734 total time=   0.0s
[CV 1/3; 58/2156] START criterion=gini, max_depth=1.0, max_features=log2, n_estimators=9
[CV 1/3; 58/2156] END criterion=gini, max_depth=1.0, max_features=log2, n_estimators=9;, score=0.734 total time=   0.0s
[CV 2/3; 58/2156] START criterion=gini, max_depth=1.0, max_features=log2, n_estimators=9
[CV 2/3; 58/2156] END criterion=gini, max_depth=1.0, max_features=log2, n_estimators=9;, score=0.734 total time=   0.0s
[CV 3/3; 58/2156] START criterion=gini, max_

...

[CV 1/3; 2103/2156] END criterion=entropy, max_depth=11.0, max_features=auto, n_estimators=45;, score=0.804 total time=   0.2s
[CV 2/3; 2103/2156] START criterion=entropy, max_depth=11.0, max_features=auto, n_estimators=45
[CV 2/3; 2103/2156] END criterion=entropy, max_depth=11.0, max_features=auto, n_estimators=45;, score=0.796 total time=   0.2s
[CV 3/3; 2103/2156] START criterion=entropy, max_depth=11.0, max_features=auto, n_estimators=45
[CV 3/3; 2103/2156] END criterion=entropy, max_depth=11.0, max_features=auto, n_estimators=45;, score=0.783 total time=   0.2s
[CV 1/3; 2104/2156] START criterion=entropy, max_depth=11.0, max_features=auto, n_estimators=46
[CV 1/3; 2104/2156] END criterion=entropy, max_depth=11.0, max_features=auto, n_estimators=46;, score=0.803 total time=   0.2s
[CV 2/3; 2104/2156] START criterion=entropy, max_depth=11.0, max_features=auto, n_estimators=46
[CV 2/3; 2104/2156] END criterion=entropy, max_depth=11.0, max_features=auto, n_estimators=46;, score=0.797 

[CV 3/3; 2116/2156] END criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=9;, score=0.774 total time=   0.1s
[CV 1/3; 2117/2156] START criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=10
[CV 1/3; 2117/2156] END criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=10;, score=0.798 total time=   0.1s
[CV 2/3; 2117/2156] START criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=10
[CV 2/3; 2117/2156] END criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=10;, score=0.779 total time=   0.1s
[CV 3/3; 2117/2156] START criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=10
[CV 3/3; 2117/2156] END criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=10;, score=0.780 total time=   0.1s
[CV 1/3; 2118/2156] START criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=11
[CV 1/3; 2118/2156] END criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=11;, score=0.799 t

[CV 1/3; 2129/2156] END criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=22;, score=0.803 total time=   0.1s
[CV 2/3; 2129/2156] START criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=22
[CV 2/3; 2129/2156] END criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=22;, score=0.790 total time=   0.1s
[CV 3/3; 2129/2156] START criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=22
[CV 3/3; 2129/2156] END criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=22;, score=0.780 total time=   0.1s
[CV 1/3; 2130/2156] START criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=23
[CV 1/3; 2130/2156] END criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=23;, score=0.805 total time=   0.1s
[CV 2/3; 2130/2156] START criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=23
[CV 2/3; 2130/2156] END criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=23;, score=0.787 

[CV 3/3; 2141/2156] END criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=34;, score=0.782 total time=   0.2s
[CV 1/3; 2142/2156] START criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=35
[CV 1/3; 2142/2156] END criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=35;, score=0.807 total time=   0.2s
[CV 2/3; 2142/2156] START criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=35
[CV 2/3; 2142/2156] END criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=35;, score=0.793 total time=   0.2s
[CV 3/3; 2142/2156] START criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=35
[CV 3/3; 2142/2156] END criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=35;, score=0.783 total time=   0.2s
[CV 1/3; 2143/2156] START criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=36
[CV 1/3; 2143/2156] END criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=36;, score=0.806 

[CV 1/3; 2154/2156] END criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=47;, score=0.807 total time=   0.2s
[CV 2/3; 2154/2156] START criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=47
[CV 2/3; 2154/2156] END criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=47;, score=0.787 total time=   0.2s
[CV 3/3; 2154/2156] START criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=47
[CV 3/3; 2154/2156] END criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=47;, score=0.785 total time=   0.2s
[CV 1/3; 2155/2156] START criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=48
[CV 1/3; 2155/2156] END criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=48;, score=0.805 total time=   0.2s
[CV 2/3; 2155/2156] START criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=48
[CV 2/3; 2155/2156] END criterion=entropy, max_depth=11.0, max_features=log2, n_estimators=48;, score=0.786 

# 以下為 K-近鄰演算法（K-Nearest Neighbors, K-NN） 模型訓練與調參

In [23]:
classifier = KNeighborsClassifier()
classifier.fit(x_train, y_train)

KNeighborsClassifier()

In [24]:
y_pred = classifier.predict(x_test)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/base.py:442: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"


In [25]:
k_range = list(range(30,35))
leaf_range = list(range(29,31))
weight_options = ['uniform']
algorithm_options = ['auto']
metric = ['minkowski']
p = list(range(2,3))
param_gridknn = dict(n_neighbors = k_range,weights = weight_options,algorithm=algorithm_options,leaf_size=leaf_range,metric = metric,p=p )
gridKNN = GridSearchCV(classifier,param_gridknn,cv=3,scoring='accuracy',verbose=10)
gridKNN.fit(x_train, y_train)
print('best score is:',str(gridKNN.best_score_))
print('best params are:',str(gridKNN.best_params_))

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3; 1/10] START algorithm=auto, leaf_size=29, metric=minkowski, n_neighbors=30, p=2, weights=uniform


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/base.py:442: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"


[CV 1/3; 1/10] END algorithm=auto, leaf_size=29, metric=minkowski, n_neighbors=30, p=2, weights=uniform;, score=0.796 total time=   0.3s
[CV 2/3; 1/10] START algorithm=auto, leaf_size=29, metric=minkowski, n_neighbors=30, p=2, weights=uniform


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/base.py:442: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"


[CV 2/3; 1/10] END algorithm=auto, leaf_size=29, metric=minkowski, n_neighbors=30, p=2, weights=uniform;, score=0.782 total time=   0.3s
[CV 3/3; 1/10] START algorithm=auto, leaf_size=29, metric=minkowski, n_neighbors=30, p=2, weights=uniform


...

[CV 1/3; 10/10] END algorithm=auto, leaf_size=30, metric=minkowski, n_neighbors=34, p=2, weights=uniform;, score=0.793 total time=   0.3s
[CV 2/3; 10/10] START algorithm=auto, leaf_size=30, metric=minkowski, n_neighbors=34, p=2, weights=uniform


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/base.py:442: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"


[CV 2/3; 10/10] END algorithm=auto, leaf_size=30, metric=minkowski, n_neighbors=34, p=2, weights=uniform;, score=0.784 total time=   0.3s
[CV 3/3; 10/10] START algorithm=auto, leaf_size=30, metric=minkowski, n_neighbors=34, p=2, weights=uniform


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/base.py:442: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"


[CV 3/3; 10/10] END algorithm=auto, leaf_size=30, metric=minkowski, n_neighbors=34, p=2, weights=uniform;, score=0.780 total time=   0.3s
best score is: 0.7870222222222222
best params are: {'algorithm': 'auto', 'leaf_size': 29, 'metric': 'minkowski', 'n_neighbors': 32, 'p': 2, 'weights': 'uniform'}


# 混淆矩陣與模型評估指標

In [15]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score,fbeta_score
cm = confusion_matrix(y_test, y_pred_dt)
print(cm)

[[833 200]
 [172 202]]


In [16]:
#Evaluate results
acc = accuracy_score(y_test, y_pred_dt )
prec = precision_score(y_test, y_pred_dt )
rec = recall_score(y_test, y_pred_dt )
f1 = f1_score(y_test, y_pred_dt )
f2 = fbeta_score(y_test, y_pred_dt, beta = 2)
print(acc,prec,rec,f1,f2 )

0.7356076759061834 0.5024875621890548 0.5401069518716578 0.5206185567010311 0.5321390937829295
